In [179]:
import pandas

## This is a helper function for the get_data

In [212]:
def flip(string):
    
    d = {'house':'face',
         'face':'house',
         'faces':'houses',
         'houses':'faces',
          'left':'right',
          'right':'left',
        'continuous':'intermittent',
        'intermittent':'continuous'}
    
    return d[string]

## This function extracts useful behavioral measures from the clean logfile

In [213]:
def get_data(fn):
    
    # Get mapping
    f = open(fn)

    lines=f.readlines()

    tactile_mapping = dict([(lines[0].split()[0], lines[0].split()[-2])])    
    tactile_mapping[flip(tactile_mapping.keys()[0])] = flip(tactile_mapping.values()[0])
    
    visual_mapping = dict([(lines[1].split()[0][:-1], lines[1].split()[-2])])
    visual_mapping[flip(visual_mapping.keys()[0])] = flip(visual_mapping.values()[0])    
    
    mapping = tactile_mapping.copy()
    mapping.update(visual_mapping)
    
    # Get data
    df = pandas.read_csv(fn, skiprows=3, sep='\t', index_col=False, header=None)
    
    df.columns = ['blocknr', 'blockcon', 'trialnr', 'houses', 'continuous', 'rightStim', 'VisStim_fn', 'ITI', 'button', 'RT', 'slow', None][:len(df.columns)]
    
    df['houses'] = df['houses'].map({'true':True, 'false':False, '':np.nan})
    df['continuous'] = df['continuous'].map({'true':True, 'false':False, '':np.nan})
    df['modality'] = df['blockcon'].map({"Tac":"Tactile", "Vis":"Visual"})

    df.ix[df.modality == "Visual", 'stimulus'] = df.ix[df.modality == "Visual", 'houses'].map({True:'house', False:'face'})
    df.ix[df.modality == "Tactile", 'stimulus'] = df.ix[df.modality == "Tactile", 'continuous'].map({True:'continuous', False:'intermittent'})


    df['stimulus_location'] = df.rightStim.map({False:'left', True:'right'})
    df['response'] = df.button.map({1:'left', 2:'right'})
    
    df['correct_answer'] = df.stimulus.map(mapping)

    df['correct'] = df['correct_answer'] == df.response

    df['congruent'] = df.stimulus_location == df.correct_answer
    df['congruency'] = df.congruent.map({True:'congruent', False:'incongruent'})


    # df['correct'] = ((df.blockcon == 'Vis') & (df.houses & (df.button == 2))) | ((df.blockcon == 'Tac') & (df.continuous & (df.button == 2)))

    # df['modality'] = df['blockcon']
    
    return df
    

## Let's also load in the raw log to be able to get correct onsets

In [214]:
df_raw = pandas.read_csv('/home/gdholla1/data/simon_amsterdam/behavior/raw/001-SimonTactile.log', sep='\t', skiprows=2)

df_clean = get_data('/home/gdholla1/data/simon_amsterdam/behavior/raw/001_SimonTact.txt')

df_clean.reset_index(inplace=True)
# df_clean.columns = list(df_clean.columns[2:]) + ['slow', None]

In [215]:
df_raw.head()

,Subject,Trial,Event Type,Code,Time,TTime,Uncertainty,Duration,Uncertainty.1,ReqTime,ReqDur,Stim Type,Pair Index
0,001,1,Picture,instruct,15466,0,1,110531,2,0,next,other,0
1,001,1,Response,2,125795,110329,1,NaN,NaN,NaN,NaN,NaN,NaN
2,001,2,Picture,instruct,128130,0,2,75066,3,0,next,other,0
3,001,2,Response,2,203102,74972,1,NaN,NaN,NaN,NaN,NaN,NaN
4,001,2,Response,2,542869,414739,1,NaN,NaN,NaN,NaN,NaN,NaN


## This code searches for the onsets at which the 8 block start

In [216]:
df_block_starts = []

n_instructs = -1 # Because the first block has two instruction screens
block = 0
last_time = 0



for index, row in df_raw.iterrows():
    
    if block < 8:
        if row['Code'] == 'instruct':
            n_instructs += 1
            print 'instruct'
            last_time = int(row.Time)

        if ((row['Event Type'] == 'Pulse') & (n_instructs == 1) & (int(row['Time']) - 10000 > last_time)):

            block += 1

            n_instructs = 0
            df_block_starts.append({'block':block,
                                    'time':row.Time})


instruct
instruct
instruct
instruct
instruct
instruct
instruct
instruct
instruct


In [217]:
df_block_starts

[{'block': 1, 'time': '608086'},
 {'block': 2, 'time': '4848126'},
 {'block': 3, 'time': '8823476'},
 {'block': 4, 'time': '12749238'},
 {'block': 5, 'time': '16812244'},
 {'block': 6, 'time': '20948898'},
 {'block': 7, 'time': '24822752'},
 {'block': 8, 'time': '28697396'}]

In [219]:
df_block_starts = pandas.DataFrame(df_block_starts).set_index('block')
df_block_starts

,time
block,
1,608086
2,4848126
3,8823476
4,12749238
5,16812244
6,20948898
7,24822752
8,28697396


## Now put blocknrs and trials in this raw data, so we can merge later

In [220]:
df_raw = df_raw[df_raw.Code == 'redfix']
df_raw['trial'] = np.arange(df_raw.shape[0])
df_raw['blocknr'] = np.floor(df_raw.trial / 54) + 1
df_raw['trialnr'] = np.floor(df_raw.trial % 54) + 1
# df_raw['block_onset'] = df_raw

In [224]:
## Merge

In [225]:
df = df_raw.merge(df_clean, on=['trialnr', 'blocknr'])

## Merge the block onsets and divide by 10000, so we get the onsets in seconds

In [226]:
df['block_onset'] = df.blocknr.map(df_block_starts.time)
df['onset'] = (df['Time'].astype(float) - df['block_onset'].astype(float)) / 10000.

In [227]:
df.head()

,Subject,Trial,Event Type,Code,Time,TTime,Uncertainty,Duration,Uncertainty.1,ReqTime,...,modality,stimulus,stimulus_location,response,correct_answer,correct,congruent,congruency,block_onset,onset
0,001,3,Picture,redfix,611188,3066,2,3067,3,2960,...,Visual,face,right,left,left,True,False,incongruent,608086,0.3102
1,001,5,Picture,redfix,651721,3067,1,3067,2,2960,...,Visual,face,right,left,left,True,False,incongruent,608086,4.3635
2,001,7,Picture,redfix,697187,3067,1,3066,3,2960,...,Visual,house,right,right,right,True,True,congruent,608086,8.9101
3,001,9,Picture,redfix,747719,3066,1,3067,2,2960,...,Visual,face,left,left,left,True,True,congruent,608086,13.9633
4,001,11,Picture,redfix,803185,3067,1,3067,2,2960,...,Visual,face,right,left,left,True,False,incongruent,608086,19.5099


In [228]:
df['subject_id'] = 1
df['run'] = df['blocknr']

In [229]:
df.to_pickle('/home/gdholla1/data/simon_amsterdam/behavior/all_data.pandas')